# 第2スクリーニング―機械学習による企業のグループ分け―

## 教師なし機械学習・クラスタ分析

In [ ]:
# 各モジュールのインポート
import csv   
import MeCab
from wordcloud import WordCloud
import matplotlib as mpl
import matplotlib.pyplot as plt 
from pathlib import Path 
import mojimoji
import unicodedata
import urllib
import pandas as pd
import numpy as np
import seaborn as sns
import collections 
import docx2txt

data_dir_path = Path('data')

In [ ]:
# 文を句点で分割するためのモジュールのインポート
import functools

from ja_sentence_segmenter.common.pipeline import make_pipeline
from ja_sentence_segmenter.concatenate.simple_concatenator import concatenate_matching
from ja_sentence_segmenter.normalize.neologd_normalizer import normalize
from ja_sentence_segmenter.split.simple_splitter import split_newline, split_punctuation

split_punc2 = functools.partial(split_punctuation, punctuations = r"。!?")
concat_tail_te = functools.partial(concatenate_matching, former_matching_rule = r"^(?P<result>.+)(て)$", remove_former_matched = False)
segmenter = make_pipeline(normalize, split_newline, concat_tail_te, split_punc2)


In [ ]:
# テキストの読み込み

# 今回は多数の企業のデータを扱うので, ファイル名を一つのエクセルファイル上で管理する
comp_fstscreening = pd.read_csv("/Users/shintaro/Downloads/2ndscreening_compfilename.csv", encoding = 'Shift_JIS') # 使用するwordファイルの名前が書かれたcsvファイルを使用します
comp_fstscreening_compname = comp_fstscreening['企業名'].tolist()
comp_fstscreening_filename = comp_fstscreening['file_name'].tolist()

# 企業名・ファイル名がそれぞれ正確に読み込まれているか確認してください(正しく読み込まれていない場合はcsvファイルの変更内容が保存されていない可能性があるので保存してから実行してください)
comp_fstscreening

In [ ]:
# 企業の番号とファイル名が入ったリストcomp_IR_tmpを作成する
comp_IR_tmp = [[]for i in range(len(comp_fstscreening_filename))]
for i in range(len(comp_IR_tmp)):
    comp_IR_tmp[i].append(i)
    comp_IR_tmp[i].append(comp_fstscreening_filename[i])

In [ ]:
# ファイルを読み込む関数read_IRを定義する
def read_IR(comp_num, file_path):
    text = docx2txt.process(file_path)
    text = text.replace('\n','').replace('\u3000','').replace('\xa0','')
    comp_IR[comp_num].append(text)

##### 各企業の統合報告書などのファイルが入ったフォルダに移動する

In [ ]:
cd /Users/shintaro/Downloads/統合報告書_2022

In [ ]:
# IRのファイルがカレントディレクトリに入っていないとFileNotFoundErrorが出るので注意してください
comp_IR = [[]for i in range(len(comp_IR_tmp))]

In [ ]:
# それぞれの企業のIRのテキストデータが入ったリストcomp_IRを作成する
for i in range(len(comp_IR_tmp)):
    if comp_IR_tmp[i][1] is np.nan:
        comp_IR[i] = []
        
    else:
        read_IR(comp_IR_tmp[i][0], comp_IR_tmp[i][1])

In [ ]:
comp_IR

In [ ]:
n = len(comp_IR) # 企業数

Text_data = comp_IR # テキストデータのリスト

### 次のセルのwords_groupに格納する単語は, グループ分けの数字の０から３の順に並べてください. 

In [ ]:
C  = []

for i in range(n):
    text = Text_data[i]
    
    # 文章を句点で分割
    List = list(segmenter(text))

    # 形態素解析
    length = len(List)
    
    def get_noun_words_from_sentence(sentence, mecab, stopword_list = []):
        return [
            x.split('\t')[0] for x in mecab.parse(sentence).split('\n') if len(x.split('\t')) > 1 and \
             '名詞' in x.split('\t')[3] and x.split('\t')[0] not in stopword_list
        ]

    mecab = MeCab.Tagger('-Ochasen -d C:/mecab-ipadic-neologd')
    nouns = [[] for i in range(length)]
    stopword_list = []

    for i in range(length):
    
        noun = get_noun_words_from_sentence(sentence = List[i], mecab = mecab, stopword_list = stopword_list)
        nouns[i] += noun
        
    len_n = len(nouns)
    
    nouns_group = []
    
    # グループ分けの関連単語を入力してください
    words_group = ['大学','外部','コーポレートアクセラレーター','アクセラレーター','ベンチャー','共同','コーポレートベンチャーキャピタル','R&d','CVC','協働','共同開発','知財','買収','M&A','ライセンス','提供戦略','開発コミュニティ','ユーザー・イノベーション','パートナーシップ','コンソーシアム','ジョイント・ベンチャー','研究開発ネットワーク','スピンオフ','プラットフォーム戦略','ランセンス・アウト']
    
    
    # テキストデータのうち, 上記の関連単語のみを残す
    
    for i in range(len_n):
        for j in range(len(nouns[i])):
            if nouns[i][j] in words_group:
                nouns_group.append(nouns[i][j])
                
    c = list(collections.Counter(nouns_group).most_common())
    C.append(c)

In [ ]:
C

In [ ]:
# Words_groupの単語の順に各単語の登場回数を並べたリストDataの作成

Data = [[] for i in range(n)]

for i in range(n):
    for j in range(len(words_group)):
        Data[i].append(0)

for i in range(n):
    for j in range(len(C[i])):
        for k in range(len(words_group)):
            if words_group[k] == C[i][j][0]:
                Data[i][k] = C[i][j][1]

In [ ]:
Data

In [ ]:
# データの保存

df = pd.DataFrame(np.array(Data),
                  columns=words_group,
                  index=comp_fstscreening_compname)

df.to_csv("ファイル名.csv", encoding='SHIft_JIS')

In [ ]:
X = Data

In [ ]:
# 元のデータのままクラスタ分析

from sklearn.cluster import KMeans
from sklearn import preprocessing
sc = preprocessing.StandardScaler()
sc.fit(X)
X_norm = sc.transform(X)
cls = KMeans(n_clusters = 4)
result = cls.fit(X_norm)
plt.scatter(X_norm[:,0], X_norm[:,1], marker = 'o', linewidth = 0.5, s = 30, c = result.labels_, alpha = 0.6)
plt.scatter(result.cluster_centers_[:,0], result.cluster_centers_[:,1], s = 70, marker = '*', c = 'red')
#plt.savefig('クラスタリング-7', dpi = 2000)
plt.show()

In [ ]:
# ２次元に次元圧縮

from matplotlib import pyplot
from numpy import where
from sklearn.manifold import TSNE

#次元圧縮のため、TSNE-Methodを用います
tsne = TSNE(n_components=2, init='pca', random_state=0)
x = tsne.fit_transform(X)

def plot_clusters(X, cluster_ids):
    for class_value in range(7):
        row_ix = where(cluster_ids == class_value)
        pyplot.scatter(X[row_ix, 0], X[row_ix, 1])
    pyplot.show() 

In [ ]:
# K-Means_Method(K平均法)
from sklearn.cluster import KMeans
kmeans_model = KMeans(n_clusters=4)
kmeans_model.fit(x)
yhat = kmeans_model.predict(x)
plot_clusters(x, yhat) 

In [ ]:
# Gaussian Mixture Model(GMM)
from sklearn.mixture import GaussianMixture
model = GaussianMixture(n_components=4)
model.fit(x)
yhat = model.predict(x)
plot_clusters(x, yhat) 

## 教師あり機械学習・分類

### 教師データの候補となる企業のグループ分け

In [ ]:
# テキストの読み込み

# 今回は多数の企業のデータを扱うので, ファイル名を一つのエクセルファイル上で管理する
model_data = pd.read_csv("/Users/shintaro/Downloads/2ndscreening_model_compfilename.csv", encoding = 'Shift_JIS') # 使用するwordファイルの名前が書かれたcsvファイルを使用します
model_data_compname = comp_model['企業名'].tolist()
model_data_filename = comp_model['file_name'].tolist()

# 企業名・ファイル名がそれぞれ正確に読み込まれているか確認してください(正しく読み込まれていない場合はcsvファイルの変更内容が保存されていない可能性があるので保存してから実行してください)
model_data

In [ ]:
# ファイルを読み込む関数read_model_data_IRを定義する
def read_model_data_IR(comp_num, file_path):
    text = docx2txt.process(file_path)
    text = text.replace('\n','').replace('\u3000','').replace('\xa0','')
    model_data_IR[comp_num].append(text)

In [ ]:
# 企業の番号とファイル名が入ったリストmodel_data_IR_tmpを作成する
model_data_IR_tmp = [[]for i in range(len(model_data_filename))]
for i in range(len(model_data_IR_tmp)):
    model_data_IR_tmp[i].append(i)
    model_data_IR_tmp[i].append(model_data_filename[i])

In [ ]:
cd /Users/shintaro/Downloads/test_tougou

In [ ]:
# IRのファイルがカレントディレクトリに入っていないとFileNotFoundErrorが出るので注意してください
model_data_IR = [[]for i in range(len(model_data_IR_tmp))]

In [ ]:
# それぞれの企業のIRのテキストデータが入ったリストmodel_data_IRを作成する
for i in range(len(model_data_IR_tmp)):
    if model_data_IR_tmp[i][1] is np.nan:
        model_data_IR[i] = []
        
    else:
        read_model_data_IR(model_data_IR_tmp[i][0], model_data_IR_tmp[i][1])

In [ ]:
N_tmp = len(model_data_IR) # 企業数

Text_data_model_data = model_data_IR # テキストデータのリスト

In [ ]:
C_model_data  = []

for i in range(N_tmp):
    text = Text_data_model_data[i]
    
    # 文章を句点で分割
    List = list(segmenter(text))

    # 形態素解析
    length = len(List)
    
    def get_noun_words_from_sentence(sentence, mecab, stopword_list = []):
        return [
            x.split('\t')[0] for x in mecab.parse(sentence).split('\n') if len(x.split('\t')) > 1 and \
             '名詞' in x.split('\t')[3] and x.split('\t')[0] not in stopword_list
        ]

    mecab = MeCab.Tagger('-Ochasen -d C:/mecab-ipadic-neologd')
    nouns = [[] for i in range(length)]
    stopword_list = []

    for i in range(length):
    
        noun = get_noun_words_from_sentence(sentence = List[i], mecab = mecab, stopword_list = stopword_list)
        nouns[i] += noun
        
    len_N = len(nouns)
    
    # テキストデータのうち, 関連単語のみを残す
    
    for i in range(len_N):
        for j in range(len(nouns[i])):
            if nouns[i][j] in words_group:
                nouns_group.append(nouns[i][j])
                
    c_model_data = list(collections.Counter(nouns_group).most_common())
    C_model_data.append(c_model_data)

In [ ]:
C_model_data

In [ ]:
# それぞれのグループに属する単語の数を入力してください
G1 = 
G2 = 
G3 = 
G4 = 

In [ ]:
# テキストマイニングによる各単語の頻出回数の計上, 及びその数値による各企業の所属グループの決定
Model_data_Score = [[] for i in range(N_tmp)]

for i in range(N_tmp):
    
    for g1 in range(G1):
        g1_list = []
        g1_list.append(C_model_data[i][g1])
        g1_sum = sum(g1_list)
        Model_data_Score[i].append(g1_sum)
        
    for g2 in range(G1:G1+G2): 
        g2_list = []
        g2_list.append(C_model_data[i][g2])
        g2_sum = sum(g2_list)
        Model_data_Score[i].append(g2_sum)
        
    for g3 in range(G2:G2+G3):
        g3_list = []
        g3_list.append(C_model_data[i][g3])
        g3_sum = sum(g3_list)
        Model_data_Score[i].append(g3_sum)
        
    for g4 in range(G3:G3+G4):
        g4_list = []
        g4_list.append(C_model_data[i][g4])
        g4_sum = sum(g4_list)
        Model_data_Score[i].append(g4_sum)

Model_data_comp_groupnum = []

for i in range(N_tmp):
    
    Max_num = max(Model_data_Score[i])
    
    for j in range(len(Model_data_Score[i])):
        if Model_data_Score[i][j] == Max_num:
            Model_data_comp_groupnum.append(j)

### 教師データの作成

In [ ]:
# テキストの読み込み

# 今回は多数の企業のデータを扱うので, ファイル名を一つのエクセルファイル上で管理する
comp_model = pd.read_csv("/Users/shintaro/Downloads/2ndscreening_model_compfilename.csv", encoding = 'Shift_JIS') # 使用するwordファイルの名前が書かれたcsvファイルを使用します
comp_model_compname = comp_model['企業名'].tolist()
comp_model_filename = comp_model['file_name'].tolist()

comp_model_groupnum = Model_data_comp_groupnum
comp_model['グループ番号'] = Model_data_comp_groupnum

# 企業名・ファイル名がそれぞれ正確に読み込まれているか確認してください(正しく読み込まれていない場合はcsvファイルの変更内容が保存されていない可能性があるので保存してから実行してください)
comp_model

In [ ]:
# ファイルを読み込む関数read_model_IRを定義する
def read_model_IR(comp_num, file_path):
    text = docx2txt.process(file_path)
    text = text.replace('\n','').replace('\u3000','').replace('\xa0','')
    comp_model_IR[comp_num].append(text)

In [ ]:
# 企業の番号とファイル名が入ったリストcomp_model_IR_tmpを作成する
comp_model_IR_tmp = [[]for i in range(len(comp_model_filename))]
for i in range(len(comp_model_IR_tmp)):
    comp_model_IR_tmp[i].append(i)
    comp_model_IR_tmp[i].append(comp_model_filename[i])

In [ ]:
cd /Users/shintaro/Downloads/test_tougou

In [ ]:
# IRのファイルがカレントディレクトリに入っていないとFileNotFoundErrorが出るので注意してください
comp_model_IR = [[]for i in range(len(comp_model_IR_tmp))]

In [ ]:
# それぞれの企業のIRのテキストデータが入ったリストcomp_model_IRを作成する
for i in range(len(comp_model_IR_tmp)):
    if comp_model_IR_tmp[i][1] is np.nan:
        comp_model_IR[i] = []
        
    else:
        read_model_IR(comp_model_IR_tmp[i][0], comp_model_IR_tmp[i][1])

In [ ]:
N = len(comp_model_IR) # 企業数

Text_data_model = comp_model_IR # テキストデータのリスト

In [ ]:
C_model  = []

for i in range(N):
    text = Text_data_model[i]
    
    # 文章を句点で分割
    List = list(segmenter(text))

    # 形態素解析
    length = len(List)
    
    def get_noun_words_from_sentence(sentence, mecab, stopword_list = []):
        return [
            x.split('\t')[0] for x in mecab.parse(sentence).split('\n') if len(x.split('\t')) > 1 and \
             '名詞' in x.split('\t')[3] and x.split('\t')[0] not in stopword_list
        ]

    mecab = MeCab.Tagger('-Ochasen -d C:/mecab-ipadic-neologd')
    nouns = [[] for i in range(length)]
    stopword_list = []

    for i in range(length):
    
        noun = get_noun_words_from_sentence(sentence = List[i], mecab = mecab, stopword_list = stopword_list)
        nouns[i] += noun
        
    len_N = len(nouns)
    
    # テキストデータのうち, 関連単語のみを残す
    
    for i in range(len_N):
        for j in range(len(nouns[i])):
            if nouns[i][j] in words_group:
                nouns_group.append(nouns[i][j])
                
    c_model = list(collections.Counter(nouns_group).most_common())
    C_model.append(c_model)

In [ ]:
C_model

In [ ]:
# Words_groupの単語の順に各単語の登場回数を並べたリストData_modelの作成

Data_model = [[] for i in range(N)]

for i in range(N):
    for j in range(len(words_group)):
        Data_model[i].append(0)

for i in range(N):
    for j in range(len(C_model[i])):
        for k in range(len(words_group)):
            if words_group[k] == C_model[i][j][0]:
                Data_model[i][k] = C_model[i][j][1]

In [ ]:
Data_model

In [ ]:
# データの保存

df = pd.DataFrame(np.array(Data_model),
                  columns=words_group,
                  index=comp_model_compname)

df.to_csv("ファイル名１.csv", encoding='SHIft_JIS')

In [ ]:
model_data = np.array(Data_model)
model_target = np.array(comp_model_groupnum)

### 機械学習を行う

#### 訓練データとテストデータに分ける

In [ ]:
from sklearn.model_selection import  ShuffleSplit
from sklearn import svm

In [ ]:
X = model_data
y = model_target

In [ ]:
data_ss = ShuffleSplit(train_size=0.6, test_size=0.4, random_state=0)

In [ ]:
train_index, test_index = next(data_ss.split(X))

In [ ]:
X_train, y_train = X[train_index], y[train_index]
X_test, y_test  = X[test_index], y[test_index]

In [ ]:
clf = svm.SVC()
clf.fit(X_train, y_train)

In [ ]:
print(clf.score(X_test, y_test))

In [ ]:
predict = clf.predict(Data)

In [ ]:
predict